In [1]:
import sys
sys.path.append('../')

from interface.fixedleg import *
from interface.floatingleg import *
from datetime import date, timedelta
from models.hullwhitesimple import *
from engines.gaussian1d import *
from index.curverateindex import *

## Step1 - simulate future curves

In [2]:
t0_pillars = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0]
t0_rates = [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
t0_rate_curve = RateCurveSimple(t0_pillars, t0_rates)

# create an instance of the hull-white model
mean_rev = 0.02
sigma = 0.045
hw = HullWhiteProcess(mean_rev, sigma, market_curve=t0_rate_curve)

# simulate short rate and discount curve 
last_grid_time = 20
time_steps = 50
hw_kernel = GaussianRateKernel1D(hw)
r_tensor, zb_tensor, grid = hw_kernel.rate_tensor(1000, last_grid_time, time_steps)

# Reshape the tensor and create a discount curve object
reshaped_tensor = tf.transpose(zb_tensor, perm=[1, 2, 0])  # Transpose the dimensions
reshaped_tensor = tf.reshape(reshaped_tensor, shape=(8, 1000, 49))
rate_curve_tensor = DiscountCurveSimple(t0_pillars, reshaped_tensor)

## Step2 - Fixed Legs

In [10]:
evaluation_date = date(2023, 6, 21)
start_date = date(2015,12,31)
fixed_schedule = [start_date]
fixed_schedule.extend([start_date + timedelta(365 *.5 *i) for i in range(1,51)])
fixed_rate = [0.04156]*(len(fixed_schedule)-1)
fixed_notional = [143857602,142099704,140995554,139343227,138170625,136487409,135100610,133235348,131633135,130088055,
                128443001,126527961,124478813,122046073,119540895,117334856,115325682,113682288,112374057,110400222,108996590,
                107427609,106277787,104934949,103826978,102318265,101011839,99241108,97427949,95062460,92643968,89488286,86306759,
                82654668,79233393,75249260,71491885,67239725,63147273,58562602,54101515,49235670,44484930,39276217,34122360,28587822,
                23186787,17327243,11581997,5627771]
# create the FixedRateLeg object
leg1 = FixedRateLeg(fixed_schedule, fixed_notional, fixed_rate)

# price is computed for all the input tensor
leg1.npv(rate_curve_tensor, evaluation_date)

<tf.Tensor: shape=(1000, 49), dtype=float64, numpy=
array([[42556051.17185441, 48871426.43313518, 44197012.32705101, ...,
        47082327.28327733, 50203283.56238392, 45617813.48510081],
       [42682533.23458554, 43747081.88581047, 44887858.31000052, ...,
        30880711.47403741, 29906445.84902079, 28838072.35467676],
       [45130379.17667498, 48016071.06403135, 49353487.524347  , ...,
        47234869.76093519, 47289676.9711016 , 41977769.66813685],
       ...,
       [42887641.76682446, 41660668.66534071, 47647500.84253135, ...,
        50453558.58637679, 52087478.6004387 , 49394758.39936744],
       [40557960.19236713, 40947235.4257664 , 44337005.35780923, ...,
        21244907.42285902, 20866626.78434666, 20672200.12219867],
       [41728489.36036718, 40484327.42200831, 43911565.90268696, ...,
        38319312.49889013, 38189307.30705784, 39758167.66449285]])>

## Step3 - Floating Legs

In [11]:
floating_schedule = [start_date]
floating_schedule.extend([start_date + timedelta(365 *.5 *i) for i in range(1,51)])

floating_notional = [0,0,0,0,0,0,0,0,0,0,
    128443001,126527961,124478813,122046073,119540895,117334856,115325682,113682288,112374057,110400222,108996590,
    107427609,106277787,104934949,103826978,102318265,101011839,99241108,97427949,95062460,92643968,89488286,86306759,
    82654668,79233393,75249260,71491885,67239725,63147273,58562602,54101515,49235670,44484930,39276217,34122360,28587822,
    23186787,17327243,11581997,5627771]

index_6m = CurveRateIndex("EUR6M", "calendar", t0_rate_curve, "6M")
spread = [0.] *(len(floating_schedule)-1)
gearing = [1.] *(len(floating_schedule)-1)

float_leg = FloatingRateLeg(floating_schedule,
                            floating_notional,
                            gearing,
                            spread,
                            index_6m)
index_6m.add_fixing(date(2022,12,29), 0.03)

# price and fixings are computed for all the input tensor
npv_float = float_leg.npv(rate_curve_tensor, evaluation_date)



## under construction -- next steps

In [12]:
# scale the tensor approach for a large portfolio, divided by currency.
# modify the pricing function such that ref_date (for future valuation) is passed by the InterestCurve object